In [2]:
import sys
sys.path.append("/Users/dmtikhonov/git_project/blondered/RecTools")

# `TwoStageModel` user guide

In [3]:
from rectools.models import PopularModel, ImplicitItemKNNWrapperModel
from implicit.nearest_neighbours import CosineRecommender
from rectools.models.rerank import TwoStageModel, CandidateGenerator, RerankerBase
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset
from sklearn.linear_model import RidgeClassifier
from pathlib import Path
import pandas as pd
from rectools import Columns

/Users/dmtikhonov/git_project/blondered/RecTools/.venv/lib/python3.10/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [7]:
%%time
!wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
!unzip -o data_original.zip
!rm data_original.zip

Archive:  data_original.zip
   creating: data_original/
  inflating: data_original/interactions.csv  
  inflating: __MACOSX/data_original/._interactions.csv  
  inflating: data_original/users.csv  
  inflating: __MACOSX/data_original/._users.csv  
  inflating: data_original/items.csv  
  inflating: __MACOSX/data_original/._items.csv  
CPU times: user 927 ms, sys: 245 ms, total: 1.17 s
Wall time: 1min 1s


In [4]:
# Prepare dataset

DATA_PATH = Path("data_original")
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = (
    pd.read_csv(DATA_PATH / 'interactions.csv', parse_dates=["last_watch_dt"])
    .rename(columns={"last_watch_dt": Columns.Datetime})
)
interactions["weight"] = 1
dataset = Dataset.construct(interactions)

In [6]:
# Prepare first stage models
first_stage = [
    CandidateGenerator(PopularModel(), 30, True, True), 
    CandidateGenerator(ImplicitItemKNNWrapperModel(CosineRecommender()), 30, True, True)
]

# Prepare splitter for selecting reranker train. Only one fold is expected!
splitter = TimeRangeSplitter("7D")

In [17]:
# Initialize TwoStageModel
# RerankerBase is not really used in final pipeline, we just didn't write the final class right now
# We can also pass negative sampler but here we are just using the default one

two_stage = TwoStageModel(first_stage, splitter, RerankerBase(RidgeClassifier()))

In [20]:
# Split dataset interactions
# Fit first stage models on history dataset
# Generate recommendations from first stage -> Get candidates for reranker
# Add targets to all candidates
# Sample negatives (here defult PerUserNegativeSampler is used) (we should probably make a public method to get data before sampling)
candidates = two_stage.get_train_with_targets_for_reranker(dataset)

In [21]:
# This is train data for boosting or any other reranker. id columns will be dropped before training
candidates.head(20)

,user_id,item_id,PopularModel_1_score,PopularModel_1_rank,ImplicitItemKNNWrapperModel_1_score,ImplicitItemKNNWrapperModel_1_rank,target
0,16090,12995,21577.0,18.0,NaN,NaN,0
1,708897,13713,NaN,NaN,0.656411,11.0,0
2,269372,512,NaN,NaN,0.104727,22.0,0
3,452915,8636,34148.0,11.0,0.142324,17.0,0
4,410073,7793,15221.0,29.0,NaN,NaN,0
5,924688,4436,16846.0,26.0,NaN,NaN,0
6,203333,11863,16231.0,29.0,0.081245,11.0,1
7,183613,4563,NaN,NaN,0.042168,12.0,0
8,1025300,15830,NaN,NaN,0.176777,7.0,0
9,336554,14431,20276.0,18.0,NaN,NaN,0


## What if we want to easily add features to candidates?

### Variant 1: from external source
Other options are:
- Get features from dataset
- Fet time-based features using_fold info from splitter
- Combine any of the above

In [5]:
from rectools.models.rerank import CandidatesFeatureCollectorBase
import typing as tp
from rectools.models.base import AnyIds

# Write custome feature collecting funcs for users, items and user/item pairs
class CustomFeatureCollector(CandidatesFeatureCollectorBase):
    def _get_user_features(
        self, users: AnyIds, dataset: Dataset, fold_info: tp.Optional[tp.Dict[str, tp.Any]], external_ids: bool
    ) -> pd.DataFrame:
        user_features = pd.read_csv(DATA_PATH / 'users.csv')
        return user_features[user_features[Columns.User].isin(users)]

In [7]:
# Now we specify our custom feature collector for TwoStageModel

two_stage = TwoStageModel(first_stage, splitter, RerankerBase(RidgeClassifier()), feature_collector=CustomFeatureCollector())

In [8]:
candidates = two_stage.get_train_with_targets_for_reranker(dataset)

In [9]:
# Now our candidates also have features for users
candidates.head(20)

,user_id,item_id,PopularModel_1_score,PopularModel_1_rank,ImplicitItemKNNWrapperModel_1_score,ImplicitItemKNNWrapperModel_1_rank,target,age,income,sex,kids_flg
0,1003127,6774,NaN,NaN,0.062984,14.0,0,age_18_24,income_20_40,Ж,0.0
1,92214,142,42877.0,8.0,0.381809,5.0,0,NaN,NaN,NaN,NaN
2,874159,896,NaN,NaN,0.125583,28.0,0,age_25_34,income_20_40,Ж,1.0
3,222588,4436,16846.0,22.0,NaN,NaN,0,age_25_34,income_20_40,М,1.0
4,90844,3784,NaN,NaN,0.398526,12.0,0,age_25_34,income_40_60,М,0.0
5,86170,13018,13372.0,29.0,NaN,NaN,0,age_35_44,income_20_40,Ж,1.0
6,80043,9728,119797.0,3.0,0.142324,4.0,0,age_45_54,income_20_40,Ж,0.0
7,888981,101,11719.0,27.0,0.347362,19.0,0,NaN,NaN,NaN,NaN
8,687876,9506,NaN,NaN,0.303832,20.0,1,age_25_34,income_0_20,Ж,0.0
9,163432,15297,180487.0,2.0,NaN,NaN,1,age_45_54,income_40_60,Ж,0.0
